<a href="https://colab.research.google.com/github/sowmen/Deepfake-Challenge-Kaggle/blob/master/MTCNN_Face_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 4.8MB/s 


In [3]:
import os, sys, time
import cv2
import numpy as np
import pandas as pd
import math
import random

from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [4]:
from mtcnn import MTCNN
mtcnn = MTCNN()

Using TensorFlow backend.


In [0]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

def scale_boxes(boxes, scale_w, scale_h):
    sb = []
    for b in boxes:
        sb.append((b[0] * scale_w, b[1] * scale_h, b[2] * scale_w, b[3] * scale_h))
    return sb

In [0]:
DATA_FOLDER = "drive/My Drive"
TEST_FOLDER = 'Annotate_Video_Stream'

In [7]:
# print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_FOLDER)))}")
print(f"Test samples: {len(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))}")

Test samples: 6


In [0]:
# train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_FOLDER)))
test_list = list(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))

In [0]:
# sample_real = "drive/My Drive/DeepFake_test/asaxgevnnp_REAL.mp4"
# sample_fake = "drive/My Drive/DeepFake_test/akxoopqjqz_FAKE.mp4"
# sample = "/content/drive/My Drive/Annotate_Video_Stream/VIDEO-2019-01-07-07-47-35.mp4"

In [0]:
def video_to_faces(video_name, input_loc, output_loc):
    """Function to extract faces from input video file
    and save them as separate images in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print("Processing : " + video_name)
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        print(frame.shape)

        result = mtcnn.detect_faces(frame)

        
        frame_copy = np.array(frame)
        frame_copy = cv2.cvtColor(frame_copy, cv2.COLOR_RGB2BGR)
        face_count = 0
        for r in range(len(result)):
            x, y, w, h = result[r]['box']

            
            

            cv2.rectangle(frame_copy, (x,y),(x+w,y+h), (255,0,0), 2)
            # frame_copy.add_patch(Rectangle((x,y),w, h, linewidth=2,edgecolor='white',facecolor='none'))
            
            face_count += 1
            
        cv2.imwrite(output_loc + "/%#05d.jpg" % ((count+1)), frame_copy)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds for conversion.\n\n" % (time_end-time_start))
            break
        

In [0]:
# !rm -r output

In [0]:
# path_out = "output_faces/" + "dqppxmoqdl.mp4" + "_" + meta_train_df.loc["dqppxmoqdl.mp4"].label
# try:
#     os.mkdir(path_out)
# except OSError:
#     print ("Creation of the directory %s failed" % path_out)
# video_to_faces("VIDEO-2019-01-07-07-47-35.mp4", "/content/drive/My Drive/Annotate_Video_Stream/VIDEO-2019-01-07-07-47-35.mp4", "/content/output")

In [10]:
#Select random videos
# random_train_selection = random.sample(train_list, 10)

for video in test_list:
    path_out = "output/" + video
    try:
        os.mkdir(path_out)
    except OSError:
        print ("Creation of the directory %s failed" % path_out)

    video_to_faces(video, os.path.join(DATA_FOLDER,TEST_FOLDER,video), os.path.join(os.getcwd(),path_out))


Streaming output truncated to the last 5000 lines.
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 3)
(352, 640, 

In [11]:
!zip -r /content/complete.zip /content/output

Streaming output truncated to the last 5000 lines.
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/01431.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/00711.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/02180.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/02128.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/02503.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/01942.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/01929.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/02316.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/02570.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/00462.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/00790.jpg (deflated 0%)
  adding: content/output/VIDEO-2019-01-07-07-47-35.mp4/00011.jpg (deflated 0%)
 

In [0]:
!cp -r complete.zip drive/My\ Drive/Annotate_Video_Stream

In [0]:
!cp -r output drive/My\ Drive/face_output